In [23]:
import numpy as np
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import csv
from moviepy.editor import *
import librosa
import tempfile
from IPython.display import Audio
from PIL import Image
import soundfile as sf
from torchinfo import summary
import os
import os.path
from ultralytics import YOLO
fd = YOLO('./models/face_yolov8m.pt')
W, H = 96, 96

In [35]:
# Extract video clip
CORPUS_PATH = '/home/lain/Merkel-Podcast-Corpus'
TIMINGS_PATH = os.path.join(CORPUS_PATH, 'timings.txt')
timings = open(TIMINGS_PATH, 'r').readlines()
clip_data = timings[0].split("|")
clip_date, clip_start, clip_end, clip_text, _ = clip_data
clip_start, clip_end = float(clip_start), float(clip_end)
clip = VideoFileClip(os.path.join(CORPUS_PATH, 'corpus', clip_date, 'video.mp4'))
audio_path = os.path.join(CORPUS_PATH, 'corpus', clip_date, 'audio.wav')
clip = clip.subclip(clip_start, clip_end)
clip = clip.subclip(2.0, 4.0)
clip.fps = 25
clip.ipython_display(width=360)


Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [28]:
frames = []
plot_frames = []
bbs = []
for i in range(50):
    t = i / clip.fps
    frame = clip.get_frame(t)
    frames.append(frame)
    pred = fd(frame, conf=0.25, device='cuda')
    if len(pred) != 1:
        print('Multiple faces detected')
        break
    bbs.append(pred[0].boxes.xyxy[0].cpu().numpy().astype(int))
    plot_frames.append(pred[0].plot())

x0, y0 = bbs[0][0], bbs[0][1]
mean_w = int(np.mean([bb[2] - bb[0] for bb in bbs]))
mean_h = int(np.mean([bb[3] - bb[1] for bb in bbs]))
face_frames = []
for idx, f in enumerate(frames):
    bb = bbs[idx]
    ff = f[y0:y0+mean_h, x0:x0+mean_w]
    ff = cv2.resize(ff, (W, H), interpolation=cv2.INTER_AREA)
    face_frames.append(ff)



0: 480x640 1 face, 15.4ms
Speed: 42.3ms preprocess, 15.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.5ms
Speed: 1.5ms preprocess, 15.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.5ms
Speed: 1.4ms preprocess, 15.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.4ms
Speed: 1.6ms preprocess, 15.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.4ms
Speed: 1.4ms preprocess, 15.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.3ms
Speed: 1.4ms preprocess, 15.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 14.5ms
Speed: 1.6ms preprocess, 14.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 14.5ms
Speed: 1.4ms preprocess, 14.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480

0: 480x640 1 face, 13.7ms
Speed: 1.4ms preprocess, 13.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.4ms preprocess, 13.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.7ms
Speed: 1.3ms preprocess, 13.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.4ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.3ms preprocess, 13.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.4ms preprocess, 13.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.2ms preprocess, 13.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.8ms
Speed: 1.3ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x6

In [26]:
frames[0].shape

(240, 320, 3)

In [29]:
video = concatenate([ImageClip(plot_frame).set_duration(1/clip.fps) for plot_frame in plot_frames])
video.ipython_display(width=360, fps=25)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [30]:
print(face_frames[0].shape)

(96, 96, 3)


In [31]:
video = concatenate([ImageClip(ff).set_duration(1/clip.fps) for ff in face_frames])
video.ipython_display(width=360, fps=25)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [17]:
with tempfile.NamedTemporaryFile(suffix='.wav') as temp_audio_file:
        audio = clip.audio.subclip(0, 2.0)
        audio = audio.set_duration(2.0)
        audio.write_audiofile(temp_audio_file.name, codec='pcm_s16le', verbose=False, write_logfile=False, logger=None)
        y, _ = librosa.load(temp_audio_file.name, sr=16000)
        print(y.shape)
        S = librosa.feature.melspectrogram(
                y=y,
                sr=16000,
                n_mels=80,
                fmax=8000,
                n_fft=800,
                hop_length=80)


(32000,)


In [16]:
S.shape

(80, 401)

In [20]:
# Convert Mel spectrogram to STFT
linear_segment = librosa.feature.inverse.mel_to_stft(S, sr=16000, n_fft=800)

# Griffin-Lim reconstruction
audio_griffin = librosa.griffinlim(linear_segment, n_iter=64, hop_length=80)  # Adjust n_iter as needed

# Save to WAV file
sf.write('/tmp/output.wav', audio_griffin, 16000)
Audio('/tmp/output.wav')